## Local tests

# prerequisites:

In [1]:
import boto3
import sagemaker

region = boto3.Session().region_name
role = 'arn:aws:iam::245582572290:role/service-role/AmazonSageMaker-ExecutionRole-20211021T134979'
default_bucket = sagemaker.session.Session().default_bucket()
training_pipeline_name = "TrainingAbalonePipeline"
source_data_bucket = "kris-predictive-maintenance-test"

## upsert pipeline

In [1]:
from mlops_utilities.actions import upsert_pipeline

In [2]:
upsert_pipeline(pipeline_module='pipelines', pipeline_package='training_pipeline', pipeline_name='mlops-utilities-upsert', role = 'arn:aws:iam::245582572290:role/service-role/AmazonSageMaker-ExecutionRole-20211021T134979')

## run pipeline

In [4]:
from mlops_utilities.actions import run_pipeline

In [5]:
run_pipeline(
    pipeline_name='TrainingAbalonePipeline',
    execution_name_prefix='latest',
    dryrun=False,
    pipe_params={
        "InputDataS3Uri": f"s3://{source_data_bucket}/abalonedata/data.csv",
        "InputLabelS3Uri": f"s3://{source_data_bucket}/abalonelabels/data.csv",
    }
)

{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:245582572290:pipeline/trainingabalonepipeline/execution/hut12fi57sog',
 'ResponseMetadata': {'RequestId': 'dc4c8916-9d91-455b-9ab4-525110158fa3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dc4c8916-9d91-455b-9ab4-525110158fa3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '123',
   'date': 'Mon, 24 Oct 2022 15:39:16 GMT'},
  'RetryAttempts': 0}}

## create endpoint

In [2]:
import boto3
import sagemaker

In [3]:
boto_sess = boto3.Session()
sm = boto_sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=boto_sess)
# EDU account
# execution_role = 'arn:aws:iam::245582572290:role/service-role/AmazonSageMaker-ExecutionRole-20211021T134979'
# main account
execution_role = 'arn:aws:iam::311638508164:role/AmazonSageMaker-ExecutionRole'
endpoint_name = 'TestEndpoint'

instance_type = 'ml.m5.large'
instance_count = 1

1. create model

In [4]:
import boto3

aws_region='us-east-1'
sagemaker_client = boto3.client('sagemaker', region_name=aws_region)
framework= "sklearn"
version = '1.0-1'
s3_bucket = 'kris-mlops-utilities-test'
model_url = 's3://sagemaker-mlops-p-8wejpwt16iyw/pipelines-m5dg8xqiu58e-TrainAbaloneModel-62auDXm3wm/output/model.tar.gz'
model_name = 'model'


In [5]:
import datetime
import os
from typing import Any, Dict, List

def _create_model_from_model_package(
        model_name: str,
        model_package_arn: str,
        execution_role: str,
        tags: List[Dict[str, str]],
) -> str:
    response = sagemaker_client.create_model(
        ModelName=model_name,
        Containers=[{'ModelPackageName': model_package_arn}],
        ExecutionRoleArn=execution_role,
        Tags=tags,
    )
    return response['ModelArn']

def _get_sagemaker_project_tags() -> List[Dict[str, str]]:
    tags = []

    project_id = os.getenv('SAGEMAKER_PROJECT_ID')
    if project_id is not None:
        tags.append({'Key': 'sagemaker:project-id', 'Value': project_id})

    project_name = os.getenv('SAGEMAKER_PROJECT_NAME')
    if project_name is not None:
        tags.append({'Key': 'sagemaker:project-name', 'Value': project_name})

    return tags

def _generate_datetime_suffix(dt_format: str = '%Y-%m-%d-%H-%M-%S') -> str:
    return datetime.datetime.now(datetime.timezone.utc).strftime(dt_format)

current_datetime_utc_str = _generate_datetime_suffix()
sagemaker_project_tags = _get_sagemaker_project_tags()

In [ ]:
model_package_arn = 'arn:aws:sagemaker:us-east-1:311638508164:model-package/test-sokolov/1'
execution_role = 'arn:aws:iam::311638508164:role/AmazonSageMaker-ExecutionRole'
model_arn = _create_model_from_model_package(
    model_name='model',
    model_package_arn=model_package_arn,
    execution_role=execution_role,
    tags=sagemaker_project_tags,
)
print('Model "%s" has been created successfully', model_arn)


2. create endpoint config

In [6]:
def _generate_data_capture_config(
        s3_destination_prefix: str,
        sampling_percentage: int = 100,
) -> Dict[str, Any]:
    return {
        'EnableCapture': True,
        'InitialSamplingPercentage': sampling_percentage,
        'DestinationS3Uri': s3_destination_prefix,
        'CaptureOptions': [{'CaptureMode': 'Input'}, {'CaptureMode': 'Output'}],  # both by default
        'CaptureContentTypeHeader': {'CsvContentTypes': ['text/csv']},
    }

In [7]:
s3_data_capture_output_prefix = None
endpoint_config_name = f'{endpoint_name}-config-{current_datetime_utc_str}'
data_capture_config = _generate_data_capture_config(s3_data_capture_output_prefix) \
    if s3_data_capture_output_prefix is not None \
    else None

In [22]:
from mlops_utilities.actions import create_endpoint
import sagemaker

boto_sess = boto3.Session()
sm = boto_sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=boto_sess)

create_endpoint(model_package_arn, sagemaker_session,
                instance_count, instance_type, endpoint_name,
                data_capture_config)

---------------!

## update endpoint

In [8]:
model_stat_s3 = 's3://sagemaker-mlops-p-sq33brxtdjtx/model-monitor/baselining/baseline-suggestion-job-2022-10-18-15-13-56-519/results/statistics.json'

In [9]:
boto_sess = boto3.Session()
sm = boto_sess.client("sagemaker")
# sagemaker_session = sagemaker.Session(boto_session=boto_sess)
sm.describe_endpoint_config(EndpointConfigName=endpoint_name)

{'EndpointConfigName': 'TestEndpoint',
 'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:311638508164:endpoint-config/testendpoint',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'ModelName': '1-2022-10-25-13-33-49-805',
   'InitialInstanceCount': 1,
   'InstanceType': 'ml.m5.large',
   'InitialVariantWeight': 1.0}],
 'CreationTime': datetime.datetime(2022, 10, 25, 16, 33, 51, 471000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '5997e3aa-cfd8-465f-b2f9-7924a081d1f9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5997e3aa-cfd8-465f-b2f9-7924a081d1f9',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '334',
   'date': 'Wed, 26 Oct 2022 12:12:40 GMT'},
  'RetryAttempts': 0}}

In [10]:
from mlops_utilities.actions import update_endpoint

In [11]:
update_endpoint(sm=sm, endpoint_name='TestEndpoint', data_capture_config=data_capture_config, model_statistics_s3_uri=model_stat_s3)

---------------!

## deploy model

In [8]:
from mlops_utilities.actions import deploy_model

data_capture_s3_uri = 's3://kris-mlops-utilities-test/'

In [9]:
boto_sess = boto3.Session()
sm = boto_sess.client("sagemaker")
model_package_group_name = 'test-Sokolov'

In [10]:
sm.list_model_packages(
        ModelApprovalStatus='Approved',
        ModelPackageGroupName=model_package_group_name,
        SortBy='CreationTime',
        SortOrder='Descending',
        MaxResults=1,
    )

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'test-Sokolov',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:311638508164:model-package/test-sokolov/1',
   'CreationTime': datetime.datetime(2022, 9, 20, 21, 28, 34, 958000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'Approved'}],
 'ResponseMetadata': {'RequestId': 'ac709ace-2a28-4ae1-85f0-cf93117d0388',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ac709ace-2a28-4ae1-85f0-cf93117d0388',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '282',
   'date': 'Wed, 26 Oct 2022 15:40:00 GMT'},
  'RetryAttempts': 0}}

In [ ]:
deploy_model(model_package_group_name, instance_type, instance_count, endpoint_name, data_capture_s3_uri)